In [1]:
%%time
# This is the code you want to measure the execution time of
for i in range(1000000):
    x = i ** 2
    
import pandas as pd
import numpy as np
import seaborn as sns
#to get rid of chained assignment errors
pd.options.mode.chained_assignment = None  # default='warn'
df = pd.read_csv('data/survey_results_public.csv')
schema_df = pd.read_csv('data/survey_results_schema.csv', index_col='qname')
df.set_index('ResponseId', inplace=True)
pd.set_option('display.max_columns',85)# applys to both df's schema and results
pd.set_option('display.max_rows',85) # applys to both df's schema and results

# looking at general Df, getting the count of total responses to put counts into perspective:
#df.columns
#schema_df.columns
#display(df.shape)
#display(schema_df.shape)
#df.head(10)
#schema_df(10)

# looking at country list, and how many have non np.NAN answers:
#print(df['Country'].unique())
#display(df['Country'].count())
#note there are 1497 NaN's


# looking at countries with the most responses, taking top 45 for now, getting total sum from those 45:
#display(df['Country'].value_counts(dropna=False,sort=True,ascending=False).head(45))
#display(np.sum(df['Country'].value_counts(dropna=True,sort=True,ascending=False).head(45)))

# Creating a df_45 DataFrame of the countries and thier respective number of partisipants.(Where I will add aggragated data): 
a = df['Country'].value_counts(dropna=True)
#display(a)
#print(type(a)) 
df_45 = a.to_frame().rename(columns={'Country': 'Count'}) #this turns series into a Df
#print(type(df_45))
# Removing unwated rows (coutries with low partisipation):
filt = df_45['Count']>316
test= df_45[filt]
#display(test)
# Setting up df_45:
df_45 = test
df_45.shape
df_45 = df_45.reset_index()
df_45 = df_45.rename(columns={'index': 'Country'})
#df_45

# Creatig a list of the countries: 
countries = test.index.tolist()
# looking at list:
#countries

# Updating original DF to only include rows from the 45 chosen countries
filt2 = df['Country'].isin(countries)
df_whole_45 = df[filt2]
#df_whole_45

# 64084 matches sum from above, should be correct
#df_whole_45.shape

CPU times: total: 2.95 s
Wall time: 3.11 s


In [ ]:
df.columns

In [2]:

### % Fully Remote:

# Group by country:
country_grp = df_whole_45.groupby(['Country'])

# Creating a series for the fractions/percentages of work locations for each country:
remote_work_perc = country_grp['RemoteWork'].value_counts(normalize=True) # normalize=True, puts it into a fraction
# Looking at the series fractions/percentages of work locations for each country:
#print((remote_work_perc))

# filter the series (remote_work_perc) DataFrame to only include rows where the value of the 'RemoteWork' column is 'Fully remote':
fully_remote_perc = remote_work_perc.loc[:, 'Fully remote'].rename('fully_remote_perc')
# Looking at the series where the value of the 'RemoteWork' column is 'Fully remote'
#print(type(fully_remote_perc))
#print((fully_remote_perc))

# Fully remote series added to df_45
# Join the series fully_remote_perc with the df_45:
df_45 = df_45.join(fully_remote_perc, on='Country')
# Create a count of participation: 
Work_Location_Count = country_grp['RemoteWork'].count().rename('Work_Location Count_nonNA')
#print(type(Work_Location_Count))
# Join the series Work_Location_Count with the df_45:
df_45 = df_45.join(Work_Location_Count, on='Country')



In [3]:
### Salary Investigaation ###:
# Setting options to keep results in floats not scientific notation:
pd.set_option('display.float_format', lambda x: f'{x:.2f}')
# Looking at currencies in the new df to see how many I need to look up:
#print(df_whole_45['Currency'].unique())
#print(df_whole_45['Currency'].value_counts())
 # NOTE: not going to use any currencies with less than 6 responses. sorry it is just for the time.

# Define dict of currency converters:
currencies_converters = {'EUR': .93, 'USD': 1, 'INR': 82.31, 'GBP': .82,'CAD': 1.37, 'BRL': 5.25, 'PLN': 4.39, 'AUD': 1.5,
                         'SEK': 10.4, 'RUB': 80.2, 'CHF': .91, 'TRY': 19.24,'ILS' : 3.58, 'CZK': 21.47, 'IRR': 42250,
                         'PKR': 288.25, 'DKK': 6.83,'MXN': 18.31, 'NOK': 10.44,'ZAR': 18.05, 'NZD':1.58, 'RON': 4.53, 'BDT': 107.52, 'HUF': 344.99 ,
                         'CNY': 6.88, 'IDR': 14959, 'UAH': 36.95 , 'JPY': 131.30, 'ARS': 211.23, 'BGN': 1.79, 'COP': 4584.75, 'PHP': 54.50,
                         'EGP': 30.90, 'NGN': 464.50, 'VND': 23467.50, 'AED': 3.67} # taken from goodle converter 3/14/2023 and 4/5

# Triming unneeded characters from 'Currency':
df_whole_45['Currency'] = df_whole_45['Currency'].str[:3]

# Function to convert currency:
def convert_currency(currency):
    if currency in currencies_converters:
        return currencies_converters[currency]
    else:
        return np.NaN  # using np.NaN instead of NaN
    
# Add new 'CurrencyConvert' column to DataFrame using apply() method:
df_whole_45['CurrencyConvert'] = df_whole_45['Currency'].apply(convert_currency)

#schema_df.loc['CompTotal']['question']
#OUTPUT: 'What is your current total compensation (salary, bonuses, and perks, before taxes and deductions)? 
#         Please enter a whole number in the box below, without any punctuation.  
#         If you are paid hourly, please estimate an equivalent weekly, monthly, or yearly salary.
#         If you prefer not to answer, please leave the box empty.'


#schema_df.loc['CompFreq']['question']
#OUTPUT: 'Is that compensation weekly, monthly, or yearly?'


# Function to adjust pay from interval to yearly:
# Note, there seems to be little consistancy with responses on salary,
#       it appears some answered with their yearly pay amount and specified the interval payments are recieved,
#       while others answered the amount they are paid at each interval(intention of questions)...
#       As well as people who blantantly provided missinformation. 

def get_interval(interval):
    if pd.isna(interval):
        return np.NaN
    elif interval == "Yearly":
        return 1
    elif interval == "Monthly":
        return 12
    elif interval == "Weekly":
        return 52
    else:
        raise ValueError("Invalid interval value: {}".format(interval))
        
        
        
# Use function - testing if the comp freq gets more results when removed:
df_whole_45['YearlyTotConvertCOMP'] = (1/df_whole_45['CurrencyConvert']) * df_whole_45['CompTotal'] * df_whole_45['CompFreq'].apply(get_interval)
# Use function - testing if the comp freq gets more results when removed:
#df_whole_45['YearlyTotConvertCOMP'] = (1/df_whole_45['CurrencyConvert']) * df_whole_45['CompTotal'] 
#trying to get rid of scientific notation again:
df_whole_45['YearlyTotConvertCOMP'] = df_whole_45['YearlyTotConvertCOMP'].apply(lambda x: float(x))


# Looking at us, italy, Iran,Nigeria...there is something very wrong with avearges!!!
#us_data = country_grp.get_group('United States of America')['YearlyTotConvertCOMP'].sort_values(ascending=False)
# Print the YearlyTotConvertCOMP values for USA
#print(us_data.head(55))
#NOTE: Looks like there are some intentional erroneous answers in the extreem,
#      as well as possibly some that indicated the incorrect currency or interval. I am removing anything over $999,999.

# Removing 'YearlyTotConvertCOMP' under$1500 and over $999,999
print('-------shape before outliers removal--------')
display(df_whole_45.shape)
filt3 = (df_whole_45['YearlyTotConvertCOMP']<999999) & (df_whole_45['YearlyTotConvertCOMP']>1500)
df_whole_45 = df_whole_45[filt3]
print('------------shape after outliers removal------------')
display(df_whole_45.shape)

# Display updated DataFrame slice:
#df_whole_45[['Country', 'Currency','CurrencyConvert', 'CompTotal', 'CompFreq','YearlyTotConvertCOMP']].sort_values(by = 'YearlyTotConvertCOMP' , ascending=True).head(75)
#df = df.dropna(axis=1).....This will remove all columns that contain at least one NaN value.
# Looking at Max and Mean
#print(df_whole_45['YearlyTotConvertCOMP'].max())
#print(df_whole_45['YearlyTotConvertCOMP'].min()) 

# Looking at mean and median
country_grp = df_whole_45.groupby(['Country'])
country_grp['YearlyTotConvertCOMP'].mean().round(0)
#pd.set_option('display.float_format', lambda x: f'{x:.2f}')
country_grp['YearlyTotConvertCOMP'].agg(['mean', 'median']).round(0)

#Salary series added to df_45
Mean_Salary = country_grp['YearlyTotConvertCOMP'].mean().round(1).rename('Mean Salary')
Median_Salary = country_grp['YearlyTotConvertCOMP'].median().round(1).rename('Median Salary')
Salary_Count_NonNA = country_grp['Age'].count().rename('Salary_Count NonNA')
df_45 = df_45.join(Mean_Salary, on='Country')
df_45 = df_45.join(Median_Salary, on='Country')
df_45 = df_45.join(Salary_Count_NonNA, on='Country')


-------shape before outliers removal--------


(64084, 80)

------------shape after outliers removal------------


(33561, 80)

In [ ]:
df_whole_45.columns


In [4]:
###  looking at YearsCode   ###:
#df_whole_45['YearsCode'].unique()

# Fix data in 'years code'
# Replacing/rewriting string responses with float values:
df_whole_45.loc[df_whole_45['YearsCode'] == 'Less than 1 year', 'YearsCode'] = 0
df_whole_45.is_copy = False
df_whole_45.loc[df_whole_45['YearsCode'] == 'More than 50 years', 'YearsCode'] = 51
df_whole_45.is_copy = False
# Ensuring Everything is a float, I had a few error messages to compete with.
df_whole_45.loc[:, 'YearsCode'] = df_whole_45['YearsCode'].astype(float)
#df_whole_45['YearsCode'] = df_whole_45['YearsCode'].astype(float)
# This is_copy = Fales is to cancel out errors/warnings I kept getting. 
df_whole_45.is_copy = False

#df_whole_45.loc[:, 'Age'] = df_whole_45['Age']
    
#checking again
#df_whole_45['YearsCode'].unique()
#df_whole_45.dropna(subset=['Age'], inplace=True)

#get Mean and Median
#display(df_whole_45['YearsCode'].mean())
#df_whole_45['YearsCode'].median()  

#Looking at results:
country_grp = df_whole_45.groupby(['Country'])
#country_grp['YearsCode'].mean().round(1)
#country_grp['YearsCode'].median().round(1)
country_grp['YearsCode'].agg(['mean', 'median']).round(1)
# checking participation 
country_grp['YearsCode'].count()


#yearsCode series added to df_45
Mean_YearsCode = country_grp['YearsCode'].mean().round(1).rename('Mean YearsCode')
Median_YearsCode = country_grp['YearsCode'].median().round(1).rename('Median YearsCode')
YearsCode_Count_NonNA = country_grp['YearsCode'].count().rename('YearsCode_Count NonNA')
df_45 = df_45.join(Mean_YearsCode, on='Country')
df_45 = df_45.join(Median_YearsCode, on='Country')
df_45 = df_45.join(YearsCode_Count_NonNA, on='Country')


C:\Users\Adison\AppData\Local\Temp\ipykernel_16796\128316929.py:11: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_whole_45.loc[:, 'YearsCode'] = df_whole_45['YearsCode'].astype(float)


In [ ]:
###  looking at age Distribution  ###:
# What is the data type? It is in ranges.
#df_whole_45['Age'].unique()
#df_whole_45['Age'].value_counts()

#looking at country group 'age'
#country_grp['Age'].value_counts().head(50)
# Note: after looking at the responses split by coutry, they all have the same most common answer of '25-34 years old'.
        # This is not helpful or intersting. I am going to replace each range with the year in the center of the range 
        # and then take the average. This will represent the spread and weight of the ages in each country.
        
def change_age(age):
    if pd.isna(age):
        return np.NaN
    elif age == "Prefer not to say":
        return np.NaN
    elif age == "Under 18 years old":
        return 16
    elif age == "18-24 years old":
        return 21
    elif age == "25-34 years old":
        return 29.5
    elif age == "35-44 years old":
        return 39.5
    elif age == "45-54 years old":
        return 49.5
    elif age == "55-64 years old":
        return 55.9
    elif age == "65 years or older":
        return 70
    else:
        raise ValueError("Invalid interval value: {}".format(age))    
        
# NOTE: Overwriting ages in df_whole_45 from ranges in to floats. This is permanant and need only run once per eval.
#       If "text Change_age already complete" appears then it has run already.


# Function to  change age, checks first if changes have been made to avoide errors.   
if df_whole_45.iloc[1]['Age'] in [16,21,29.5,39.5,49.5,55.9,70]:
        print("Change_age already complete")
else:df_whole_45.loc[:, 'Age'] = df_whole_45['Age'].apply(change_age)


#looking at age distribution after change, looks good
# df_whole_45['Age'].unique()
# df_whole_45['Age'].value_counts()
# country_grp['Age'].value_counts().head(50)

# Getting average by country:
#country_grp['Age'].mean().round(1)

# Recording participation: 
country_grp = df_whole_45.groupby(['Country'])
country_grp['Age'].count()

#Age series added to df_45
Mean_Age = country_grp['Age'].mean().round(1).rename('Mean Age')
Age_Count_NonNA = country_grp['Age'].count().rename('Age_Count NonNA')
df_45 = df_45.join(Mean_Age, on='Country')
df_45 = df_45.join(Age_Count_NonNA, on='Country')


In [5]:
### Education level ###:
country_grp = df_whole_45.groupby(['Country'])
#country_grp['EdLevel'].unique()
#country_grp['EdLevel'].value_counts().head(40)
#country_grp = df_whole_45.groupby(['Country'])

Top_Education_per_country = country_grp['EdLevel'].apply(lambda x: x.value_counts().index[0])

# Replace original education levels with simplified versions:
replace_dict = {
    'Some college/university study without earning a degree': 'Some college',
    'Bachelor’s degree (B.A., B.S., B.Eng., etc.)': 'Bachelor’s',
    'Master’s degree (M.A., M.S., M.Eng., MBA, etc.)': 'Master’s'
}

# education level added to df_45 
Top_Education_per_country = Top_Education_per_country.replace(replace_dict).rename('Most_Common ED_Level')
df_45 = df_45.join(Top_Education_per_country, on='Country')
Ed_level_Count_NonNA= country_grp['EdLevel'].count().rename('Ed_level Count NonNA')
df_45 = df_45.join(Ed_level_Count_NonNA, on='Country')
print(Top_Education_per_country)

#updating df_whole_45 EdLevel
df_whole_45['EdLevel'] = df_whole_45['EdLevel'].replace(replace_dict)




Country
Argentina                                               Some college
Australia                                                 Bachelor’s
Austria                                                     Master’s
Bangladesh                                                Bachelor’s
Belgium                                                   Bachelor’s
Brazil                                                    Bachelor’s
Bulgaria                                                  Bachelor’s
Canada                                                    Bachelor’s
China                                                     Bachelor’s
Colombia                                                  Bachelor’s
Czech Republic                                              Master’s
Denmark                                                     Master’s
Egypt                                                     Bachelor’s
Finland                                                     Master’s
France                    

In [6]:
# get top listed language used, NOTE--This is not very interesting, nearly all JavaScript and HTML/CSS

#print(df_whole_45['LanguageHaveWorkedWith'].unique())
#df_whole_45['LanguageHaveWorkedWith'] = df_whole_45['LanguageHaveWorkedWith'].str.replace(',', ';')
#country_grp = df_whole_45.groupby(['Country'])
#country_grp = country_grp.apply({'LanguageHaveWorkedWith': lambda x: x.str.split(',')})
#print(country_grp['LanguageHaveWorkedWith'].value_counts())


df_whole_45['LanguageHaveWorkedWith'] = df_whole_45['LanguageHaveWorkedWith'].str.replace(',', ';')
#country_grp = df_whole_45.groupby('Country')['LanguageHaveWorkedWith'].apply(lambda x: x.str.split(';').explode().value_counts())
#print(country_grp.head(75))

country_grp = df_whole_45.groupby('Country')['LanguageHaveWorkedWith'].apply(lambda x: x.str.split(';').explode().value_counts())
top_languages = country_grp.groupby(level=0).nlargest(3)
top_languages.head(75)




#df_whole_45['LanguageHaveWorkedWith'] = df_whole_45['LanguageHaveWorkedWith'].str.replace(';', ',')

#country_grp = df_whole_45.groupby(['Country'])
#country_grp = country_grp.apply({'LanguageHaveWorkedWith': lambda x: x.str.split(',')})
#country_grp
#country_grp = country_grp.explode('LanguageHaveWorkedWith')
#counts = country_grp['LanguageHaveWorkedWith'].value_counts()

#country_grp['LanguageHaveWorkedWith'].value_counts()
#print(counts)


# make a copy of the DataFrame
#df_whole_45_copy = df_whole_45.copy()



# Split the values in the 'CodingActivities' column by the delimiter ';'
#country_grp = country_grp.agg({'LanguageHaveWorkedWith': lambda x: x.str.split([';',','])})



# Use the explode() method to create a new row for each value in the resulting lists


# Use the value_counts() method to count the occurrence of each value
#counts = country_grp['LanguageHaveWorkedWith'].value_counts()

#print(counts)

Country                       Country                                 
Argentina                     Argentina                     JavaScript     194
                                                            HTML/CSS       154
                                                            SQL            142
Australia                     Australia                     JavaScript     549
                                                            HTML/CSS       449
                                                            SQL            449
Austria                       Austria                       JavaScript     257
                                                            HTML/CSS       223
                                                            SQL            218
Bangladesh                    Bangladesh                    JavaScript     156
                                                            HTML/CSS       110
                                                            

In [ ]:
This is an example of a table with a "multi-index" or "hierarchical index" which is used in pandas, a popular data analysis library in Python.

Think of it as a big table where each row represents a person who filled out a survey, and each column represents a piece of information they provided. In this case, the columns are "Country," "Language," and "Number of people."

The first column of the table is called "Country," which tells us which country the people who filled out the survey come from. The second column is also called "Country," but it is nested underneath the first column. This second "Country" column tells us the same thing as the first "Country" column, but it's used to group the data based on country.

So, for example, if we look at the row that says "Argentina" in the first "Country" column and "JavaScript" in the second "Country" column, we see that 194 people from Argentina listed JavaScript as a language they know.

Similarly, if we look at the row that says "Australia" in the first "Country" column and "HTML/CSS" in the second "Country" column, we see that 449 people from Australia listed HTML/CSS as a language they know.

This nested structure allows us to group and analyze the data in different ways, based on the level of detail we need.

In [7]:

# Replace ';' with ','
df['LanguageHaveWorkedWith'] = df['LanguageHaveWorkedWith'].str.replace(';', ',')

# Split the 'LanguageHaveWorkedWith' column by ',' and explode the resulting list
df = df.assign(LanguageHaveWorkedWith=df['LanguageHaveWorkedWith'].str.split(',')).explode('LanguageHaveWorkedWith')

# Group by 'Country' and 'LanguageHaveWorkedWith', and count the occurrences
df = df.groupby(['Country', 'LanguageHaveWorkedWith']).size().reset_index(name='Count')

# Sort by 'Country' and 'Count' in descending order
df = df.sort_values(['Country', 'Count'], ascending=[True, False])

# Preview the resulting dataframe
print(df.tail(75))

       Country LanguageHaveWorkedWith  Count
4975  Viet Nam                 Erlang      4
4986  Viet Nam                   LISP      4
4998  Viet Nam                    SAS      4
4989  Viet Nam                  OCaml      3
4970  Viet Nam                Clojure      2
4976  Viet Nam                     F#      2
4977  Viet Nam                Fortran      2
5009     Yemen                     C#      8
5014     Yemen               HTML/CSS      8
5012     Yemen                   Dart      7
5026     Yemen                    SQL      7
5015     Yemen                   Java      6
5016     Yemen             JavaScript      6
5022     Yemen                    PHP      5
5024     Yemen                 Python      4
5017     Yemen                 Kotlin      3
5027     Yemen                  Swift      3
5006     Yemen               Assembly      2
5007     Yemen             Bash/Shell      2
5008     Yemen                      C      2
5010     Yemen                    C++      2
5013     Y

In [ ]:
df_whole_45

In [ ]:
df_whole_45
sns.scatterplot(x = 'YearlyTotConvertCOMP', y = 'YearsCode', data = df_whole_45, hue = 'EdLevel' )

In [ ]:
# get top listed job (MainBranch):
#This is less intersting when devided by country, I want to make a pie chart of participation by listed responsibilites/job title.
df['DevType'].value_counts().head(40)
 
# make a copy of the DataFrame
df_copy = df.copy()

# drop NaN values
df_copy = df_copy.dropna(subset=['DevType'])

# Split the values in the 'CodingActivities' column by the delimiter ';'
df_copy['DevType'] = df_copy['DevType'].str.split(';')

# Use the explode() method to create a new row for each value in the resulting lists
df_copy = df_copy.explode('DevType')

# Use the value_counts() method to count the occurrence of each value
counts = df_copy['DevType'].value_counts()

# Divide each count by the total number of responses and multiply by 100 to get the percentage
percentages = (counts / len(df_copy)) * 100

print(percentages.head(45))

In [ ]:
#taking a look at df_45
df_45

In [ ]:
sns.scatterplot(x = 'Median Salary', y = 'Mean Age', data = df_45, hue = "Most_Common ED_Level", size = 'Median YearsCode' )

In [ ]:
# make a copy of the DataFrame
df_copy = df.copy()

# Replace NaN values with an empty string
df_copy['CodingActivities'] = df_copy['CodingActivities'].fillna('')

# Split the values in the 'CodingActivities' column by the delimiter ';'
df_copy['CodingActivities'] = df_copy['CodingActivities'].str.split(';')

# Use the explode() method to create a new row for each value in the resulting lists
df_copy = df_copy.explode('CodingActivities')

# Use the value_counts() method to count the occurrence of each value
counts = df_copy['CodingActivities'].value_counts()

print(counts)